# Post Process

In [32]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import seaborn as sns
from pathlib import Path
from IPython.display import display
import pickle
import random
# from env_utils import read_pso_search_files, detect_convergence
%matplotlib inline

from data_utils import find_match_from_seach_collector, get_unique_search_df, get_time_diff_df

In [33]:
agents_data = {}

base_path = Path("../../DataPack/output-BehAVExplor")

ADS_name = ["apollo","ba","interfuser","tfpp","highway"]
# ADS_name = ["apollo"]

if 'BehAVExplor' in str(base_path):
    BX = True
else:
    BX = False

for scenario_name in os.listdir(base_path):
    scenario_path = os.path.join(base_path, scenario_name)
    if os.path.isdir(scenario_path):
        for agent_name in os.listdir(scenario_path):

            # Optinal: select one agent
            if agent_name not in ADS_name:
                continue
            agent_path = os.path.join(scenario_path, agent_name)
            if os.path.isdir(agent_path):
                if agent_name not in agents_data:
                    agents_data[agent_name] = []
                for experiment_name in os.listdir(agent_path):

                    name_splits = experiment_name.split("_")
                    if len(name_splits) < 4:
                        pass
                    else:
                        # delete the last _w
                        old_experiment_name = experiment_name
                        random_5_digits = random.randint(10000, 99999)
                        if BX:
                            experiment_name = f"{random_5_digits}"
                        else:
                            experiment_name = f"{random_5_digits}_{name_splits[1]}_{name_splits[2]}"
                        # rename the foleder
                        os.rename(os.path.join(agent_path, old_experiment_name), os.path.join(agent_path, experiment_name))


                    # iterate each file in the experiment folder
                    experiment_path = os.path.join(agent_path, experiment_name)
                    if os.path.isdir(experiment_path):
                        for file_name in os.listdir(experiment_path):
                            if not file_name.endswith(".csv") or not file_name.startswith("search"):
                                # delete the file
                                os.remove(os.path.join(experiment_path, file_name))
                            else:
                                # read the file
                                this_df = pd.read_csv(os.path.join(experiment_path, file_name))
                                
                                exclude_cols = ["loss", "time"]
                                if BX:
                                    exclude_cols.append("collision_status")
                                # remove cols in exclude_cols

                                # if this_df has exclude_cols
                                if set(exclude_cols).issubset(this_df.columns):
                                    this_df = this_df.drop(columns=exclude_cols)


                                for index, row in this_df.iterrows():
                                    search_id = str(row["search_id"])
                                    if '.' in search_id:
                                        search_id = search_id.split(".")[1]
                                    this_df.at[index, "search_id"] = search_id

                                # save the file
                                this_df.to_csv(os.path.join(experiment_path, file_name), index=False)

                                if BX:
                                    # rename the file name to search_bx.csv
                                    os.rename(os.path.join(experiment_path, file_name), os.path.join(experiment_path, "search_bx.csv"))



                    

